In [1]:
import psycopg2 as pg
import sqlalchemy
import petl as etl
from datetime import datetime
import pytz
from petl import datetimeparser

In [3]:
isodatetime = datetimeparser('%Y-%m-%d %H:%M:%S')

In [4]:
conn = pg.connect("host=172.20.0.2 dbname=testcase user=airflow password=airflow")

In [49]:
table1 = etl.fromcsv(source = '/home/jovyan/work/test1.csv',delimiter = ';')

In [51]:
table2 = etl.convert(table1,'id',int)

In [54]:
table3.lookall()

+----+-------------+-----------------------+-----------------------+
| id | data        | modi_date             | load_date             |
+====+=============+=======================+=======================+
|  1 | 'sdsdsd'    | '2012-01-07 00:00:00' | '2021-06-08 09:13:13' |
+----+-------------+-----------------------+-----------------------+
|  2 | 'sdsdjfhsd' | '2012-01-07 00:01:00' | '2021-06-08 09:13:13' |
+----+-------------+-----------------------+-----------------------+
|  3 | 'x'         | '2012-01-07 00:00:00' | '2021-06-08 09:13:13' |
+----+-------------+-----------------------+-----------------------+
|  4 | 'd'         | '2012-01-08 00:00:00' | '2021-06-08 09:13:13' |
+----+-------------+-----------------------+-----------------------+
|  5 | 'sdd'       | '2012-01-07 00:00:00' | '2021-06-08 09:13:13' |
+----+-------------+-----------------------+-----------------------+
|  6 | 'ssd'       | '2012-01-07 00:00:00' | '2021-06-08 09:13:13' |
+----+-------------+--------------

In [53]:
now = datetime.now()
now_str = now.strftime("%Y-%d-%m %H:%M:%S")
now_str
table3 = etl.addfield(table2,'load_date',now_str)

In [55]:
table4 = etl.select(table3,lambda rec: rec.id >=3 and rec.id < 7)
table4

id,data,modi_date,load_date
3,x,2012-01-07 00:00:00,2021-06-08 09:13:13
4,d,2012-01-08 00:00:00,2021-06-08 09:13:13
5,sdd,2012-01-07 00:00:00,2021-06-08 09:13:13
6,ssd,2012-01-07 00:00:00,2021-06-08 09:13:13


In [59]:
etl.todb(table =  table4 ,dbo = conn, tablename = "test_csvpetl_3", schema = 'olist', create = True, drop=True)

In [62]:
table5 = etl.convert(table4,"load_date",lambda d : isodatetime(d))

In [65]:
conn.close()

При опции в todb create = true таблица создается на лету, поэтому тип дейттайм питона не встанет в постгрес, так как там нету этого типа (есть timestamp). Нам надо будет заранее создавать таблицу в постгрес, тогда дейттайм спокойно встанет на таймстемп постгрес